In [1]:
import matplotlib.pyplot as plt
from python import OptimGSCS, model
import numpy as np
from copy import deepcopy
import logging
log_file = 'logs_compression_test.log'
open(log_file, 'w').close()
logging.basicConfig(filename=log_file, level=logging.INFO)

from importlib import reload

In [2]:
m = model.model_pruning('poids/model0.keras')
a = m.current_weights

In [3]:
def f(a):
    m.update_weights(deepcopy(a))
    return(-m.percentage_pruned)

def g(a):
    m.update_weights(deepcopy(a))
    return(m.evaluate_pruned())

first_center = OptimGSCS.random_weights(m.current_weights, f, g)

In [ ]:
reload(OptimGSCS)
print(first_center.performance, first_center.norm)
experience = OptimGSCS.serie_random_weights(tolerance=0.87, center=first_center, multiple_dimension=True)
repeat = 0
logging.info(" | ".join([str("repeat"), str("experience.min_weight.performance"), str("experience.min_weight.norm")]))
logging.info(" | ".join([str(repeat), str(experience.min_weight.performance), str(experience.min_weight.norm)]))
min_norm = first_center.norm
while repeat <1000:
    repeat +=1
    experience.extend()
    logging.info(" | ".join([str(repeat), str(experience.min_weight.performance), str(experience.min_weight.norm)]))
    if not (repeat % 50):
        print()
        print(repeat, experience.min_weight.performance, experience.min_weight.norm)
        print(repeat, experience.last_center_gs.performance, experience.last_center_gs.norm)        

0.9145 -0.098

50 0.8753 -0.197
50 0.8703 -0.195

100 0.8729 -0.2
100 0.8703 -0.195

150 0.8764 -0.201
150 0.8703 -0.195

200 0.8727 -0.206
200 0.8736 -0.203

250 0.874 -0.212
250 0.8711 -0.211

300 0.8702 -0.234
300 0.8711 -0.233

350 0.871 -0.24
350 0.871 -0.24

400 0.87 -0.241
400 0.871 -0.24

450 0.87 -0.241
450 0.871 -0.24

500 0.87 -0.241
500 0.871 -0.24


In [ ]:
repeat_list, performance_list, norm_list = [], [], []
for line in open(log_file).readlines():
    if 'repeat' in line:
        continue
    line_s = line[10:].split(' | ')
    repeat_list.append(int(line_s[0]))
    performance_list.append(float(line_s[1]))
    norm_list.append(float(line_s[2]))

In [ ]:
plt.xlabel('# steps')
plt.ylabel('% pruned')
plt.plot(repeat_list, -np.array(norm_list))

In [ ]:
plt.xlabel('# steps')
plt.ylabel('Performance')
plt.plot(repeat_list[:400], performance_list[:400])

In [ ]:
steps = list(range(len(repeat_list)))[1:]
steps_cs = [i for i in steps if i%2]
steps_gs = [i for i in steps if not(i%2)]
diminution_cs = [int(norm_list[i-1] > norm_list[i]) for i in steps_cs]
diminution_gs = [int(norm_list[i-1] > norm_list[i]) for i in steps_gs]

In [ ]:
plt.xlabel('step number')
plt.ylabel('diminution at step i')
plt.plot(steps_ls[:200], diminution_cs[:200],'.')
plt.plot(steps_rs[:200], diminution_gs[:200],'.')
plt.legend(['GS','CS'])